## Pytorch Implementation of Feed-Forward Neural Networks

In [14]:
import torch
from torch import nn
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [68]:
# Read in the dataset
x = []
y = []
for line in open("../datasets/iris.csv"):
    data = line.rstrip('\n').split(',')
    x.append(data[0:4])
    y.append(data[4])

# convert feature lists to a numpy array
x = np.array(x, dtype = float)
# one-hot encode the label (3 classes) - a manual approach
label = np.zeros((len(y), 3))
for i in range(len(y)):
    if y[i] == 'Iris-setosa':
        label[i][0] = 1
    if y[i] == 'Iris-versicolor':
        label[i][1] = 1
    if y[i] == 'Iris-virginica':
        label[i][2] = 1
label = np.array(label, dtype = float)

# training-validation split (70% training)
x_train, x_val, label_train, label_val = train_test_split(x, label, test_size = 0.3)
print(x_train.shape, x_val.shape, label_train.shape, label_val.shape)

(105, 4) (45, 4) (105, 3) (45, 3)


In [69]:
# create DataLoader objects to read data for pytorch
# The following DataLoader takes numpy array as inputs
class IrisData(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype = torch.float32)
        self.y = torch.tensor(y, dtype = torch.float32)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [70]:
train_loader = DataLoader(IrisData(x_train, label_train), shuffle = True, batch_size = 32)
val_loader = DataLoader(IrisData(x_val, label_val), shuffle = False)

In [71]:
# Now, define the feed-forward neural network architecture
class FFNet(nn.Module):
    # first need to implement the init that specifies all layers
    def __init__(self):
        super(FFNet, self).__init__()
        # Relu hidden layer that takes 4 features and have 8 hidden units
        self.l1 = nn.Linear(4,8)
        self.relu = nn.ReLU()
        # then the output layer before softmax
        # softmax is applied as part of loss function
        self.l2 = nn.Linear(8,3)
    
    # next need to implement a forward method that takes an input through the network
    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        return x

In [75]:
# initialize the model and set training parameters
model = FFNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
epochs = 100

In [76]:
# start training:
for epoch in range(epochs):
    # training mode
    model.train()
    for batch_id, (x_train, y_train) in enumerate(train_loader):
        y_pred = model(x_train)
        train_loss = criterion(y_pred, y_train)
        # backprop
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
    
    # validation mode
    model.eval()
    with torch.no_grad():
        for (x_val, y_val) in val_loader:
            y_pred = model(x_val)
            val_loss = criterion(y_pred, y_val)
    
    # print training and val loss
    print("Epoch:", epoch, "training loss = ", train_loss, "val loss = ", val_loss)

Epoch: 0 training loss =  tensor(1.0274, grad_fn=<DivBackward1>) val loss =  tensor(0.4749)
Epoch: 1 training loss =  tensor(1.3813, grad_fn=<DivBackward1>) val loss =  tensor(0.6420)
Epoch: 2 training loss =  tensor(1.0155, grad_fn=<DivBackward1>) val loss =  tensor(0.7456)
Epoch: 3 training loss =  tensor(0.9442, grad_fn=<DivBackward1>) val loss =  tensor(0.7734)
Epoch: 4 training loss =  tensor(0.9235, grad_fn=<DivBackward1>) val loss =  tensor(0.8066)
Epoch: 5 training loss =  tensor(1.0432, grad_fn=<DivBackward1>) val loss =  tensor(0.7893)
Epoch: 6 training loss =  tensor(0.8383, grad_fn=<DivBackward1>) val loss =  tensor(0.7727)
Epoch: 7 training loss =  tensor(0.9549, grad_fn=<DivBackward1>) val loss =  tensor(0.7508)
Epoch: 8 training loss =  tensor(0.9607, grad_fn=<DivBackward1>) val loss =  tensor(0.7458)
Epoch: 9 training loss =  tensor(0.9344, grad_fn=<DivBackward1>) val loss =  tensor(0.7330)
Epoch: 10 training loss =  tensor(1.0057, grad_fn=<DivBackward1>) val loss =  te

Epoch: 89 training loss =  tensor(0.4602, grad_fn=<DivBackward1>) val loss =  tensor(0.1145)
Epoch: 90 training loss =  tensor(0.2790, grad_fn=<DivBackward1>) val loss =  tensor(0.1100)
Epoch: 91 training loss =  tensor(0.2854, grad_fn=<DivBackward1>) val loss =  tensor(0.1048)
Epoch: 92 training loss =  tensor(0.4526, grad_fn=<DivBackward1>) val loss =  tensor(0.1043)
Epoch: 93 training loss =  tensor(0.4199, grad_fn=<DivBackward1>) val loss =  tensor(0.1047)
Epoch: 94 training loss =  tensor(0.4377, grad_fn=<DivBackward1>) val loss =  tensor(0.1075)
Epoch: 95 training loss =  tensor(0.3346, grad_fn=<DivBackward1>) val loss =  tensor(0.1056)
Epoch: 96 training loss =  tensor(0.5178, grad_fn=<DivBackward1>) val loss =  tensor(0.1048)
Epoch: 97 training loss =  tensor(0.5123, grad_fn=<DivBackward1>) val loss =  tensor(0.1075)
Epoch: 98 training loss =  tensor(0.3799, grad_fn=<DivBackward1>) val loss =  tensor(0.1038)
Epoch: 99 training loss =  tensor(0.2889, grad_fn=<DivBackward1>) val 

In [80]:
# Make predictions
input = torch.tensor([5.1,3.5,1.4,0.2])
raw = model(input)
# get predicted probs
nn.Softmax(dim = 0)(raw)

tensor([0.9423, 0.0541, 0.0036], grad_fn=<SoftmaxBackward0>)